In [1]:
import math
import shutil

import pandas as pd
from constants import DATA_DIR

from astrofit.utils import AsteroidLoader

In [2]:
DAMIT_FILES = DATA_DIR / "damit" / "files"
ASTEROIDS_DIR = DATA_DIR / "asteroids"
ASTEROID_MODELS_CSV = DATA_DIR / "damit" / "tables" / "asteroid_models.csv"

In [3]:
if not ASTEROIDS_DIR.exists():
    ASTEROIDS_DIR.mkdir()

In [4]:
asteroid_loader = AsteroidLoader(DATA_DIR)

In [5]:
df = asteroid_loader.asteroids_df
df

,number,name,comment,created,modified,is_numbered
id,,,,,,
101,2,Pallas,NaN,NaN,2019-05-31 11:22:28,False
102,3,Juno,NaN,NaN,2010-04-15 15:54:09,False
103,5,Astraea,NaN,NaN,2010-04-15 15:54:09,False
104,6,Hebe,NaN,NaN,2010-04-15 15:54:09,False
105,7,Iris,NaN,NaN,2019-05-31 11:23:12,False
...,...,...,...,...,...,...
10860,566,Stereoskopia,NaN,2023-10-03 11:28:29,2023-10-03 11:28:29,False
10861,738,Alagasta,NaN,2023-10-03 11:28:29,2023-10-03 11:28:29,False
10862,138852,NaN,NaN,2024-02-20 13:13:16,2024-02-20 13:13:16,False


In [6]:
models_df = pd.read_csv(ASTEROID_MODELS_CSV, index_col=0)
models_df.head()

,asteroid_id,lambda,beta,period,yorp,jd0,phi0,lsm,lsm_p1,lsm_p2,...,visual_albedo,visual_albedo_err,craters_angle,craters_surface_density,quality_flag,nonconvex,version,comment,created,modified
id,,,,,,,,,,,,,,,,,,,,,
101,101,35.0,-12.0,7.813230,NaN,2.433828e+06,0.0,LSL,0.1,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2011-02-20,NaN,NaN,2017-05-18 21:23:47
102,101,32.0,-11.0,7.813220,NaN,2.433828e+06,0.0,LSL,0.1,NaN,...,NaN,NaN,NaN,NaN,4.0,1,2017-05-26,NaN,NaN,2017-05-26 13:18:37
103,102,103.0,27.0,7.209531,NaN,2.434753e+06,0.0,LSL,0.1,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2011-03-26,NaN,NaN,2011-04-11 11:13:18
104,103,126.0,40.0,16.800610,NaN,2.436250e+06,0.0,LSL,0.1,20.46,...,NaN,NaN,NaN,NaN,NaN,0,2011-03-26,size from AO is 110+/-14 km,NaN,2013-07-23 11:07:20
106,104,340.0,42.0,7.274471,NaN,2.436537e+06,0.0,LSL,0.1,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2011-03-26,NaN,NaN,2011-04-11 11:14:09


In [7]:
def is_nan(x: float | str):
    return x != x

In [8]:
problematic = {}
for asteroid_id in df.index:
    print(f"Processing asteroid {asteroid_id:4} ... ", end="")
    asteroid_dir = DAMIT_FILES / f"asteroid_{asteroid_id}"
    if not asteroid_dir.exists():
        problematic[asteroid_id] = "missing asteroid dir"
        print(f"Missing asteroid dir for {asteroid_id}!\n")
        continue

    lc_file = asteroid_dir / "lc.json"
    if not lc_file.exists():
        problematic[asteroid_id] = "missing lightcurve file"
        print(f"missing lightcurve file!\n")
        continue

    spins = {}
    for model_dir in asteroid_dir.iterdir():
        if not model_dir.is_dir():
            continue

        if not model_dir.name.startswith("model_"):
            print(f"WARNING - unexpected dir: {model_dir}!\t")
            continue

        spin_names = ["IAUspin", "IAUspin.txt", "spin.txt"]
        for name in spin_names:
            spin_file = model_dir / name
            if spin_file.exists():
                break
        else:
            problematic[asteroid_id] = "missing spin file"
            print(f"missing spin file!\n")
            continue

        model_no = model_dir.name.split("_")[-1]
        spins[model_no] = model_dir / spin_file

    period = models_df[models_df["asteroid_id"] == asteroid_id]["period"]
    if period.empty:
        problematic[asteroid_id] = "missing period"
        print(f"missing period!\n")
        continue

    period = period.values[-1]

    asteroid_name, = df[df.index == asteroid_id]["name"]
    if is_nan(asteroid_name):
        problematic[asteroid_id] = "missing name"
        print(f"missing name!\n")
        continue

    asteroid_dir = ASTEROIDS_DIR / f"{asteroid_name}"
    if asteroid_dir.exists():
        print(f"already processed!\n")
        continue
    else:
        asteroid_dir.mkdir()

    with open(asteroid_dir / "period.txt", "w") as f:
        f.write(str(period))
    
    shutil.copy(lc_file, asteroid_dir / "lc.json")
    for model_no, spin_file in spins.items():
        shutil.copy(spin_file, asteroid_dir / f"spin_{model_no}")

    print("done!")

Processing asteroid  101 ... done!
Processing asteroid  102 ... done!
Processing asteroid  103 ... done!
Processing asteroid  104 ... done!
Processing asteroid  105 ... done!
Processing asteroid  106 ... done!
Processing asteroid  107 ... done!
Processing asteroid  108 ... done!
Processing asteroid  109 ... done!
Processing asteroid  110 ... done!
Processing asteroid  111 ... done!
Processing asteroid  112 ... done!
Processing asteroid  113 ... done!
Processing asteroid  114 ... done!
Processing asteroid  115 ... done!
Processing asteroid  116 ... done!
Processing asteroid  117 ... done!
Processing asteroid  118 ... done!
Processing asteroid  119 ... done!
Processing asteroid  120 ... done!
Processing asteroid  121 ... done!
Processing asteroid  122 ... done!
Processing asteroid  123 ... done!
Processing asteroid  124 ... done!
Processing asteroid  125 ... done!
Processing asteroid  126 ... done!
Processing asteroid  127 ... done!
Processing asteroid  128 ... done!
Processing asteroid 

In [9]:
from collections import Counter


In [10]:
problematic

{2509: 'missing period',
 2519: 'missing name',
 2520: 'missing name',
 2522: 'missing name',
 2524: 'missing name',
 2528: 'missing name',
 2531: 'missing name',
 2536: 'missing name',
 2539: 'missing name',
 2540: 'missing name',
 2542: 'missing name',
 2546: 'missing name',
 2548: 'missing name',
 2549: 'missing name',
 2554: 'missing name',
 2555: 'missing name',
 2565: 'missing name',
 2577: 'missing name',
 2578: 'missing name',
 2592: 'missing name',
 2606: 'missing name',
 2611: 'missing name',
 2612: 'missing name',
 2614: 'missing name',
 2615: 'missing name',
 2618: 'missing name',
 2619: 'missing name',
 2623: 'missing name',
 2625: 'missing name',
 2627: 'missing name',
 2628: 'missing name',
 2629: 'missing name',
 2631: 'missing name',
 2632: 'missing name',
 2633: 'missing name',
 2634: 'missing name',
 2641: 'missing name',
 2649: 'missing name',
 2654: 'missing name',
 2658: 'missing name',
 2659: 'missing name',
 2660: 'missing name',
 2661: 'missing name',
 2665: 'm

In [11]:
Counter(problematic.values())

Counter({'missing name': 5694, 'missing period': 1})

In [12]:
asteroid_loader = AsteroidLoader(DATA_DIR)